In [1]:
import pandas as pd

import requests
import json
import pandas as pd
from itertools import chain
import time
import concurrent.futures
from requests.exceptions import HTTPError
import json
import ast
import re
import zipfile
import string

import matplotlib.pyplot as plot
import numpy as np

!pip install transformers
!pip install keyphrasetransformer
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

#download and define embedding model
model = SentenceTransformer('allenai-specter')

  Preparing metadata (setup.py) ... done
  Created wheel for keyphrasetransformer: filename=keyphrasetransformer-0.0.2-py3-none-any.whl size=4180 sha256=92efb2b38d43fb5e3867c28d5bb12c7eb4809828d18838840b2fabc720bcbac1
  Stored in directory: /root/.cache/pip/wheels/a6/84/70/8a043e66f4f5dd346129170b3729f80985226b4aace0f2795b
Successfully built keyphrasetransformer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/331 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/462k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Input data**

In [2]:
# Connect notebook to Drive (left panel) by mounting drive

people = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_people_year0.csv')
#people_1 = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/people.csv')
grants = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_grants.csv')
papers = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/papers.csv')
companies = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_companies.csv')
patents = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/baylor_inventors.csv')

sample_inventors = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/sample_inventors_with_startups_baylor.csv')
sample_inventors=sample_inventors.fillna(0)

papers_new = pd.read_csv('/content/drive/MyDrive/portal_innovation/data/unrolled_papers_df.csv')

In [ ]:

# for id in people['portal_index_id']:
#     paper_min = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
#     people.loc[people['portal_index_id'].astype(str).str.contains(str(id)), 'earliest_consecutive_year'] = paper_min + 1

#     # if id =='A5032502969':
#     #   print(paper_min)
#     #   break
# people.to_csv('/content/drive/MyDrive/portal_innovation/data/baylor_people_year0.csv')

In [ ]:
# people[people['portal_index_id']=='A5032502969']

,name,portal_index_id,institution_id,institution,past_known_id,works_count,citation_count,h_index,mean_citedness,other_names,past_known,earliest_consecutive_year
241,Charles Y. Lin,A5032502969,02pttbw34,Baylor College of Medicine,"['https://openalex.org/I181547552', 'https://o...",180,15074,43,8.481481,"['Lin Charles', 'Charles Y. Lin', 'Charles Lin']","['I181547552', 'I2801539370']",2017.0


In [ ]:
#patents

In [ ]:
#patents[patents['portal_index_id']=='A5020956671']
#patents[(patents['portal_index_id'].astype(str).str.contains(str('A5029257596')))]
# patents['publication_year'].min()

In [3]:
grants[(grants['portal_index_id'].astype(str).str.contains(str('A5057135409')))]

,fiscal_year,project_num,organization,award_type,activity_code,award_amount,is_active,principal_investigators,contact_pi_name,spending_categories,project_start_date,project_end_date,abstract_text,project_title,budget_start,budget_end,funding_mechanism,direct_cost_amt,indirect_cost_amt,portal_index_id
2418,2018,1T32CA203690-01A1,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",1,T32,194523.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","FUQUA, SUZANNE AW","[120, 132]",2018-09-15T12:09:00Z,2023-08-31T12:08:00Z,Abstract\nImprovements in the outcome for wome...,Translational Breast Cancer Research Training ...,2018-09-15T12:09:00Z,2019-08-31T12:08:00Z,"Training, Institutional",180114.0,14409.0,"['A5057135409', 'A5054255756']"
2469,2017,1U01CA214125-01,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",1,U01,1452478.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 320, 3070]",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2017-06-07T12:06:00Z,2018-05-31T12:05:00Z,Non-SBIR/STTR,1203661.0,248817.0,['A5057135409']
2669,2020,2P50CA186784-06,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",2,P50,2137830.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","ELLIS, MATTHEW J",NaN,2014-09-19T12:09:00Z,2025-07-31T12:07:00Z,Abstract\nLack of progress in curing metastati...,Translational Research in Breast Cancer,2020-08-01T12:08:00Z,2021-07-31T12:07:00Z,Research Centers,1356232.0,813437.0,['A5057135409']
11765,2019,5T32CA203690-02,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,T32,202856.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","FUQUA, SUZANNE AW","[120, 132]",2018-09-15T12:09:00Z,2023-08-31T12:08:00Z,Abstract\nImprovements in the outcome for wome...,Translational Breast Cancer Research Training ...,2019-09-01T12:09:00Z,2020-08-31T12:08:00Z,"Training, Institutional",187830.0,15026.0,"['A5057135409', 'A5054255756']"
11766,2020,5T32CA203690-03,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,T32,211871.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","FUQUA, SUZANNE AW","[120, 132, 3641, 3584, 3724]",2018-09-15T12:09:00Z,2023-08-31T12:08:00Z,Abstract\nImprovements in the outcome for wome...,Translational Breast Cancer Research Training ...,2020-09-01T12:09:00Z,2021-08-31T12:08:00Z,"Training, Institutional",211542.0,16923.0,"['A5057135409', 'A5054255756']"
12220,2018,5U01CA214125-02,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,U01,1413657.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 320, 3070]",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2018-06-01T12:06:00Z,2019-05-31T12:05:00Z,Non-SBIR/STTR,1385104.0,248760.0,['A5057135409']
12221,2019,5U01CA214125-03,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,U01,1351343.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 320, 3070]",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2019-06-01T12:06:00Z,2020-05-31T12:05:00Z,Non-SBIR/STTR,1378091.0,215803.0,['A5057135409']
12222,2020,5U01CA214125-04,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,U01,1403852.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 3641, 320...",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2020-06-01T12:06:00Z,2021-05-31T12:05:00Z,Non-SBIR/STTR,1203729.0,200123.0,['A5057135409']


# **Sample Inventor portal_ID**

In [ ]:
df = pd.DataFrame(people)
items = []

for inventor_name in sample_inventors['Inventor Name']:
  #print(inventor_name)
  if inventor_name in df['name'].values:
      # If it exists, retrieve the corresponding row and append it to the 'items' list
      items.append(df[df['name'] == inventor_name]['portal_index_id'].values[0])
  elif df['other_names'].str.contains(inventor_name).any():
      items.append(df[df['other_names'].str.contains(inventor_name)]['portal_index_id'].values[0])
  else:
      # If it doesn't exist, append a placeholder value (such as None)
      print('NaN')
      items.append(None)

print(items)

['A5068589021', 'A5057135409', 'A5037254064', 'A5032502969', 'A5069533756', 'A5066250865', 'A5064526325', 'A5064093388', 'A5075327892', 'A5020956671', 'A5029257596', 'A5034531546', 'A5054721642', 'A5004306472', 'A5002585034', 'A5082327934', 'A5077859814', 'A5022368721']


In [ ]:
# def sample(data):

#   df = pd.DataFrame(data)

#   sample_data = []

#   for id in items:
#     #print(inventor_name)
#     if id in df['portal_index_id'].values:
#         # If it exists, retrieve the corresponding row and append it to the 'items' list
#         sample_data.append(df[df['portal_index_id'] == id].values[0])
#     elif df['portal_index_id'].astype(str).str.contains(str(id)).any():
#         sample_data.append(df[df['portal_index_id'].str.contains(id)].values[0])
#     else:
#         # If it doesn't exist, append a placeholder value (such as None)
#         sample_data.append([ ])

#   sample_data = [data.columns] + sample_data

#   #sample_data.columns.tolist() = df.columns.tolist()

#   sample_data = pd.DataFrame(sample_data)

#   sample_data.columns = sample_data.iloc[0]
#   sample_data = sample_data[1:]

#   return sample_data

# **pick out sample inventor's data**

In [ ]:
def sample(data):

    df = pd.DataFrame(data)
    sample_data = []

    for id in items:
        # Filter the DataFrame for rows where 'portal_index_id' matches the id
        matches = df[df['portal_index_id'].astype(str).str.contains(str(id))]

        # If matches exist, append all matching rows to sample_data
        if not matches.empty:
            sample_data.extend(matches.values.tolist())

        else:
            sample_data.append([ ])

    # Add the column names as the first row of sample_data
    sample_data = [df.columns.tolist()] + sample_data

    # Create a DataFrame from sample_data
    sample_df = pd.DataFrame(sample_data)

    # Set the column names from the first row
    sample_df.columns = sample_df.iloc[0]

    # Remove the first row (which contains column names) from sample_df
    sample_df = sample_df[1:]

    return sample_df


In [ ]:
people_sample = []
grants_sample = []
papers_sample = []
companies_sample = []
patents_sample = []


people_sample = sample(people)
grants_sample = sample(grants)
papers_sample = sample(papers_new)
companies_sample = sample(companies)
patents_sample = sample(patents)


In [ ]:
sample_inventors

,Inventor Name,OpenAlex ID,Startups,Startup value (mm$)
0,Xi Chen,A5068589021,RootPath,0
1,Matthew J. Ellis,A5057135409,Bioclassifier,0
2,Michael T. Lewis,A5037254064,Tvardi Therapeutics,$200
3,Charles Y. Lin,A5032502969,76Bio,$30
4,Luis Martin Diaz,A5069533756,Personal Genome Diagnostics,$575
5,Joshua Cohen,A5066250865,Amylyx (NAS: AMLX),"$1,073"
6,Yuanyuan Zhou,A5064526325,Pattern Insight,$12
7,David G. Edwards,A5064093388,automed,$58
8,Cory M. Smith,A5075327892,GC Therapeutics,$140
9,Ann M. Leen,A5020956671,AlloVir (NAS: ALVR),"$1,065"


In [ ]:
grants_sample.columns

Index(['fiscal_year', 'project_num', 'organization', 'award_type',
       'activity_code', 'award_amount', 'is_active', 'principal_investigators',
       'contact_pi_name', 'spending_categories', 'project_start_date',
       'project_end_date', 'abstract_text', 'project_title', 'budget_start',
       'budget_end', 'funding_mechanism', 'direct_cost_amt',
       'indirect_cost_amt', 'portal_index_id'],
      dtype='object', name=0)

# **pick out company related data**

In [4]:
def sample(data):

    df = pd.DataFrame(data)
    sample_data = []

    for id in companies['portal_index_id']:
        # Filter the DataFrame for rows where 'portal_index_id' matches the id
        matches = df[df['portal_index_id'].astype(str).str.contains(str(id))]

        # If matches exist, append all matching rows to sample_data
        if not matches.empty:
            sample_data.extend(matches.values.tolist())
        else:
            sample_data.append([ ])

    # Add the column names as the first row of sample_data
    sample_data = [df.columns.tolist()] + sample_data

    # Create a DataFrame from sample_data
    sample_df = pd.DataFrame(sample_data)

    # Set the column names from the first row
    sample_df.columns = sample_df.iloc[0]

    # Remove the first row (which contains column names) from sample_df
    sample_df = sample_df[1:]


    # Iterate over the DataFrame
    index_to_drop = []
    for i in range(len(sample_df) - 1):
        if sample_df.iloc[i].equals(sample_df.iloc[i + 1]):
            # Mark the index of the row to be dropped
            index_to_drop.append(i + 1)

    # Drop the marked rows
    sample_df = sample_df.drop(index_to_drop)

    sample_df = sample_df.reset_index(drop=True)

    return sample_df


In [5]:
grants['project_end_date'].values[0][0:4]

'2004'

In [6]:
people_sample    = []
grants_sample    = []
papers_sample    = []
companies_sample = []
patents_sample   = []

people_sample = sample(people)
grants_sample = sample(grants)
papers_sample = sample(papers)
papers_sample_new = sample(papers_new)
#companies_sample = sample(companies)
patents_sample = sample(patents)

In [7]:
for i in range(len(companies)):
    if pd.isna(companies['Most Recent Valuation (MM)'].loc[i]):
        companies.loc[companies['Most Recent Valuation (MM)'].isna(), 'Most Recent Valuation (MM)'] = companies['Raised to Date ($MM)']

In [14]:
grants_sample

,fiscal_year,project_num,organization,award_type,activity_code,award_amount,is_active,principal_investigators,contact_pi_name,spending_categories,project_start_date,project_end_date,abstract_text,project_title,budget_start,budget_end,funding_mechanism,direct_cost_amt,indirect_cost_amt,portal_index_id
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,2018,1T32CA203690-01A1,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",1,T32,194523.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","FUQUA, SUZANNE AW","[120, 132]",2018-09-15T12:09:00Z,2023-08-31T12:08:00Z,Abstract\nImprovements in the outcome for wome...,Translational Breast Cancer Research Training ...,2018-09-15T12:09:00Z,2019-08-31T12:08:00Z,"Training, Institutional",180114.0,14409.0,"['A5057135409', 'A5054255756']"
2,2017,1U01CA214125-01,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",1,U01,1452478.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 320, 3070]",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2017-06-07T12:06:00Z,2018-05-31T12:05:00Z,Non-SBIR/STTR,1203661.0,248817.0,['A5057135409']
3,2020,2P50CA186784-06,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",2,P50,2137830.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","ELLIS, MATTHEW J",NaN,2014-09-19T12:09:00Z,2025-07-31T12:07:00Z,Abstract\nLack of progress in curing metastati...,Translational Research in Breast Cancer,2020-08-01T12:08:00Z,2021-07-31T12:07:00Z,Research Centers,1356232.0,813437.0,['A5057135409']
4,2019,5T32CA203690-02,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,T32,202856.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","FUQUA, SUZANNE AW","[120, 132]",2018-09-15T12:09:00Z,2023-08-31T12:08:00Z,Abstract\nImprovements in the outcome for wome...,Translational Breast Cancer Research Training ...,2019-09-01T12:09:00Z,2020-08-31T12:08:00Z,"Training, Institutional",187830.0,15026.0,"['A5057135409', 'A5054255756']"
5,2020,5T32CA203690-03,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,T32,211871.0,False,"[{'profile_id': 1955335, 'first_name': 'MATTHE...","FUQUA, SUZANNE AW","[120, 132, 3641, 3584, 3724]",2018-09-15T12:09:00Z,2023-08-31T12:08:00Z,Abstract\nImprovements in the outcome for wome...,Translational Breast Cancer Research Training ...,2020-09-01T12:09:00Z,2021-08-31T12:08:00Z,"Training, Institutional",211542.0,16923.0,"['A5057135409', 'A5054255756']"
6,2018,5U01CA214125-02,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,U01,1413657.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 320, 3070]",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2018-06-01T12:06:00Z,2019-05-31T12:05:00Z,Non-SBIR/STTR,1385104.0,248760.0,['A5057135409']
7,2019,5U01CA214125-03,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,U01,1351343.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 320, 3070]",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2019-06-01T12:06:00Z,2020-05-31T12:05:00Z,Non-SBIR/STTR,1378091.0,215803.0,['A5057135409']
8,2020,5U01CA214125-04,"{'org_name': 'BAYLOR COLLEGE OF MEDICINE', 'ci...",5,U01,1403852.0,False,"[{'profile_id': 1965067, 'first_name': 'STEVEN...","ELLIS, MATTHEW J","[108, 120, 132, 1393, 176, 180, 276, 3641, 320...",2017-06-07T12:06:00Z,2022-05-31T12:05:00Z,Gene expression-based approaches to breast can...,MICROSCALED PROTEOGENOMICS FOR CANCER CLINICAL...,2020-06-01T12:06:00Z,2021-05-31T12:05:00Z,Non-SBIR/STTR,1203729.0,200123.0,['A5057135409']
9,None,None,None,None,None,None,None,None,None,No

In [20]:

variables_0_10 = pd.DataFrame(index=people_sample['portal_index_id'],columns = ['paper frequency','paper citation frequency','paper citation per paper',\
                                                                                'author position','is_corresponding',\
                                                                                'grant number', 'grant money','grant money per grant','grant series_u','grant series_r',\
                                                                                'grant series_t','grant series_p','grant duration per grant' ,  \
                                                                                'patents citations frequency' ,'patents frequency', \
                                                                                'Startup value (mm$)'])

nn = variables_0_10.shape[1]

for id in people_sample['portal_index_id']:

        variables_0_10['Startup value (mm$)'].loc[id] = companies[companies['portal_index_id'].astype(str).str.contains(str(id))]['Raised to Date ($MM)'].values[0]

        ################ paper frequency ############
        #paper_max = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).max()
        #paper_min = (papers[papers['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
        #print(paper_max,paper_min)
        freq = 0
        paper_min = people_sample[people_sample['portal_index_id'].astype(str).str.contains(str(id))]['earliest_consecutive_year'].values.astype(int)
        if paper_min[0] > 2013:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2013) ]['publication_year'].dropna().count() / (2023 - paper_min + 1)
        else:
            freq = papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+10) ]['publication_year'].dropna().count() / 10
        variables_0_10['paper frequency'].loc[id] = freq


        ################ paper citation ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2013) ]['forward_citations'].sum()) / (2023 - paper_min + 1)
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+10) ]['forward_citations'].sum()) / 10
            #print(id,paper_min)
        variables_0_10['paper citation frequency'].loc[id] = freq

        ################# paper citation per paper ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] >= 2013) ]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&(papers['publication_year'] >= 2013) ]['publication_year'].dropna().count()
        else:
            freq = float(papers[
                        (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                        (papers['publication_year'] < paper_min[0]+10) ]['forward_citations'].sum() ) / papers[ (papers['portal_index_id'].astype(str).str.contains(str(id)))&(papers['publication_year'] >= 2013) ]['publication_year'].dropna().count()
            #print(id,paper_min)
        variables_0_10['paper citation per paper'].loc[id] = freq


        ################# grant number frequency ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].dropna().count() ) / (2023 - paper_min + 1)
            # print(float(grants[
            #                 (grants['portal_index_id'].astype(str).str.contains(str(id))) &
            #                 (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].dropna().count()  ))
            # print(freq)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['project_start_date'].dropna().count() ) / 10
            #print(id,paper_min)
        variables_0_10['grant number'].loc[id] = freq


        # if paper_min[0] > 2013:
        #     print(float(grants[
        #                   (grants['portal_index_id'].astype(str).str.contains(str(id))) &
        #                   (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].dropna().count()  ))
        #     print(grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
        #                     (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_end_date'].str[:4].astype(int).sum())

        ################# grant money frequency ############
        freq = 0
        if paper_min[0] > 2013:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].sum() ) / (2023 - paper_min + 1)
        else:
            freq = float(grants[
                        (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                        (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['award_amount'].sum() ) / 10
            #print(id,paper_min)
        variables_0_10['grant money'].loc[id] = freq


        ################# grant money per grant ############
        freq = 0
        if paper_min[0] > 2013:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['fiscal_year'].dropna().count() )
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['fiscal_year'].dropna().count() )
            if num != 0:
              freq = float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['award_amount'].sum() ) / float(grants[
                          (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                          (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['fiscal_year'].dropna().count() )
              #print(id,paper_min)
        variables_0_10['grant money per grant'].loc[id] = freq

        ################# grant series ############
        if paper_min[0] > 2013:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            uu = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'U')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_u'].loc[id] = uu


        if paper_min[0] > 2013:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            rr = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'R')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_r'].loc[id] = rr


        if paper_min[0] > 2013:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            tt = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'T')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_t'].loc[id] = tt

        if paper_min[0] > 2013:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['award_amount'].dropna().count()
        else:
            pp = grants[
                (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                (grants['activity_code'].str[0] == 'P')  &
                (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10)  ]['award_amount'].dropna().count()
        variables_0_10['grant series_p'].loc[id] = pp


        ################# grant duration per grant ############
        freq = 0
        if paper_min[0] > 2013:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['fiscal_year'].dropna().count() )
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int) >= 2013) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        else:
            num = float(grants[
                      (grants['portal_index_id'].astype(str).str.contains(str(id))) &
                      (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['fiscal_year'].dropna().count() )
            #print(num)
            if num != 0.0:
                end = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                            (grants['project_start_date'].str[:4].astype(int) < paper_min[0]+10) ]['project_end_date'].str[:4].astype(int).sum()
                start = grants[(grants['portal_index_id'].astype(str).str.contains(str(id))) &
                              (grants['project_start_date'].str[:4].astype(int)< paper_min[0]+10) ]['project_start_date'].str[:4].astype(int).sum()
                freq = ( end - start )/ num
        variables_0_10['grant duration per grant'].loc[id] = freq




        ################# author position ############
        if paper_min[0] > 2013:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year'] >= 2013) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year'] >= 2013) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year'] >= 2013) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() / (2023 - paper_min + 1)
        else:
            first = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'first')  &
                (papers['publication_year']  < paper_min[0]+10) ]
            middle = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'middle') &
                (papers['publication_year']  < paper_min[0]+10) ]
            last = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['author_position'] == 'last')   &
                (papers['publication_year']  < paper_min[0]+10) ]
            aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count() /10
        variables_0_10['author position'].loc[id] = aupo



        ################# is corresponding ############

        if paper_min[0] > 2013:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year'] >= 2013) ]
            isco =  croT['publication_year'].dropna().count()
            variables_0_10['is_corresponding'].loc[id] = isco / (2023 - paper_min + 1)
        else:
            croT = papers[
                (papers['portal_index_id'].astype(str).str.contains(str(id))) &
                (papers['is_corresponding'] == True)&
                (papers['publication_year']< paper_min[0]+10)]
            isco =  croT['publication_year'].dropna().count()
            variables_0_10['is_corresponding'].loc[id] = isco / 10


        ################ patents frequency ############
        freq = 0
        if paper_min[0] > 2013:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] >= 2013) ]['publication_year'].dropna().count() / (2023 - paper_min + 1)
        else:
            freq = patents[
                        (patents['portal_index_id'].astype(str).str.contains(str(id))) &
                        (patents['publication_year'] < paper_min[0]+10) ]['publication_year'].dropna().count() / 10
            #print(id,paper_min)
        variables_0_10['patents frequency'].loc[id] = freq

        # ################ patents citations ############

        if paper_min[0] > 2013:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year'] >= 2013)]['npl_cite_count'].sum()) / (2023 - paper_min + 1)
            variables_0_10['patents citations frequency'].loc[id] = patcita
        else:
            patcita = float(patents.loc[
                (patents['portal_index_id'].astype(str).str.contains(str(id)))&
                (papers['publication_year']  < paper_min[0]+10)]['npl_cite_count'].sum()) / 10
            variables_0_10['patents citations frequency'].loc[id] = patcita


#sample_inventors_new = sample_inventors
#sample_inventors_new.index = people_sample['portal_index_id']

#variables_0_10['Startup value (mm$)'] = sample_inventors_new['Startup value (mm$)']

#variables_0_10['Total score'] = (variables_0_10.iloc[:,0:nn]).sum(axis=1)/nn #(scores.iloc[:,0]+scores.iloc[:,1]+scores.iloc[:,2]+scores.iloc[:,3]+scores.iloc[:,4]+5*scores.iloc[:,5])/nn #

# for col in ['Startup value (mm$)']:
#     min_val = variables_0_10[col].min()
#     max_val = variables_0_10[col].max()
#     variables_0_10[col] = (variables_0_10[col] - min_val) / (max_val - min_val)


variables_0_10 = variables_0_10.round(3)

variables_0_10=variables_0_10.fillna(0)

variables_0_10['Startup value (mm$)'] = variables_0_10['Startup value (mm$)'].str.replace(',', '').str.replace('$', '').str.strip().fillna(0).astype(float)


In [21]:
variables_0_10

,paper frequency,paper citation frequency,paper citation per paper,author position,is_corresponding,grant number,grant money,grant money per grant,grant series_u,grant series_r,grant series_t,grant series_p,grant duration per grant,patents citations frequency,patents frequency,Startup value (mm$)
portal_index_id,,,,,,,,,,,,,,,,
A5032502969,2.857142857142857,151.71428571428572,53.100000,62.0,0.42857142857142855,0.0,0.0,0.000000,0,0,0,0,0.000000,0.0,0.14285714285714285,10.01
A5020956671,2.9,313.4,84.702703,106.1,0.6,0.0,0.0,0.000000,0,0,0,0,0.000000,1.9,0.0,430.60
A5066250865,0.25,62.25,249.000000,3.0,0.0,0.0,0.0,0.000000,0,0,0,0,0.000000,0.0,1.0,389.58
A5054721642,0.9,9.7,48.500000,33.2,0.5,0.0,0.0,0.000000,0,0,0,0,0.000000,0.0,0.0,51.56
A5002585034,0.8,89.7,25.628571,24.0,0.0,0.0,0.0,0.000000,0,0,0,0,0.000000,0.0,0.0,0.00
A5064093388,1.0,3.0,3.000000,3.0,0.0,0.0,0.0,0.000000,0,0,0,0,0.000000,0.0,4.0,10.37
A5057135409,0.3,167.3,20.402439,13.0,0.0,0.0,0.0,0.000000,0,0,0,0,0.000000,0.1,0.0,0.00
A5022368721,0.6,43.4,108.500000,14.2,0.1,0.0,0.0,0.000000,0,0,0,0,0.000000,0.0,0.0,20.00
A5004306472,3.0,7.0,2.333333,9.0,0.0,0.0,0.0,0.000000,0,0,0,0,0.000000,0.0,1.0,57.50


# **pick out sample startups < 5mm**

In [ ]:
sample_inventors['Startup value (mm$)']=sample_inventors['Startup value (mm$)'].str.replace(',', '').str.replace('$', '').str.strip().fillna(0).astype(int)

In [ ]:
sample_inventors[sample_inventors['Startup value (mm$)'] > 5]
sample_inventors[sample_inventors['Startup value (mm$)'] < 5]

In [ ]:
def timeseries(sample_data,year_header):

    sampledata = sample_data
    df = pd.DataFrame(sampledata)

    start = int(df[year_header].min())
    end = int(df[year_header].max())
    index_range = range(start, end+1)
    # time_data = pd.DataFrame(columns=[])  # Example columns, replace with your own


    # time_data.index = index_range

    time_data = {}


    for id in items:

        person  = pd.DataFrame(columns=range(df.shape[1]),index=index_range)#pd.DataFrame(columns=range(df.shape[1]))  # Example columns, replace with your own
        # person.index = index_range

        person.columns = df.columns

        # Filter the DataFrame for rows where 'portal_index_id' matches the id
        matches = df[df['portal_index_id'].astype(str).str.contains(str(id))]

        for year in index_range:
            if year in matches[year_header].values:
                person.loc[year] = matches[matches[year_header] == year].iloc[0]

        #person=person.fillna(0)

        new_dict = {str(id):person}

        time_data.update(new_dict)

    return time_data



papers_time = timeseries(papers_sample,'publication_year')
companies_time = timeseries(companies_sample,'Year Founded')
patents_time = timeseries(patents_sample,'publication_year')

# **simple model test**

In [ ]:

scores = pd.DataFrame(index=items,columns = ['paper frequency','20 year paper freq', \
                                             '10y paper growth rate','20y paper growth rate', \
                                             'author position','is_corresponding',   \
                                             'forward_citations','patents frequency',  \
                                             'patents citations' , \
                                             'Total score','Startup value (mm$)'])

nn = scores.shape[1]

for id in items:

    ################ paper frequency ############
    paper_max = (papers_sample[papers_sample['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).max()
    paper_min = (papers_sample[papers_sample['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
    paperfreq = float(people_sample[people_sample['portal_index_id'].astype(str).str.contains(str(id))]['works_count'].iloc[0] / (paper_max - paper_min + 1))
    scores['paper frequency'].loc[id] = paperfreq

    ################ paper frequency the last 20 years ############
    filtered_data = papers_sample[
        (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (papers_sample['publication_year'] > 2004)]
    twtypapfreq = filtered_data['publication_year'].dropna().count() / 20
    scores['20 year paper freq'].loc[id] = twtypapfreq

    # ################ 10y paper growth rate ############
    # pfive = papers_sample[
    #     (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
    #     (papers_sample['publication_year'] > 2014)  &  (papers_sample['publication_year'] < 2020) ]
    # afive = papers_sample[
    #     (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
    #     (papers_sample['publication_year'] > 2019)  ]
    # tengr = ( pfive['publication_year'].dropna().count() - pfive['publication_year'].dropna().count() ) / pfive['publication_year'].dropna().count()

    # ################ 20y paper growth rate ############


    ################ author position ############
    first = papers_sample[
        (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (papers_sample['author_position'] == 'first')]
    middle = papers_sample[
        (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (papers_sample['author_position'] == 'middle')]
    last = papers_sample[
        (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (papers_sample['author_position'] == 'last')]
    aupo = 5 * first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count()
    scores['author position'].loc[id] = aupo

    ################ is corresponding ############
    croT = papers_sample[
        (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (papers_sample['is_corresponding'] == True)]
    croF = papers_sample[
        (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (papers_sample['is_corresponding'] == False)]
    isco = 5 * croT['publication_year'].dropna().count() +  1 * croF['publication_year'].dropna().count()
    scores['is_corresponding'].loc[id] = isco

    ################ citation ############
    filtered_data = papers_sample[
        (papers_sample['portal_index_id'].astype(str).str.contains(str(id)))]
    cita = float(filtered_data['forward_citations'].sum()) / float(people_sample[people_sample['portal_index_id'].astype(str).str.contains(str(id))]['works_count'].iloc[0])
    scores['forward_citations'].loc[id] = cita

    # ################ author position ############
    # first = papers_sample[
    #     (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
    #     (papers_sample['author_position'] == 'first')]
    # middle = papers_sample[
    #     (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
    #     (papers_sample['author_position'] == 'middle')]
    # last = papers_sample[
    #     (papers_sample['portal_index_id'].astype(str).str.contains(str(id))) &
    #     (papers_sample['author_position'] == 'last')]
    # aupo = 5*first['publication_year'].dropna().count() + 3 * middle['publication_year'].dropna().count() + 1 * last['publication_year'].dropna().count()
    # scores['author position'].loc[id] = aupo

    ################ patents frequency ############
    patents_sample = pd.DataFrame(patents_sample)
    patents_max = (patents_sample[patents_sample['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).max()
    patents_min = (patents_sample[patents_sample['portal_index_id'].astype(str).str.contains(str(id))]['publication_year']).min()
    active = patents_sample[
        (patents_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (patents_sample['legal_status'] == 'ACTIVE') ]
    pending = patents_sample[
        (patents_sample['portal_index_id'].astype(str).str.contains(str(id))) &
        (patents_sample['legal_status'] == 'PENDING') ]
    #print(active['publication_year'].dropna().count(),pending['publication_year'].dropna().count())
    patentsfreq = float( (5 * active['publication_year'].dropna().count() + 1 * pending['publication_year'].dropna().count()) / (patents_max-patents_min+1))
    scores['patents frequency'].loc[id] = patentsfreq


    ################ patents citations ############
    filtered_data = patents_sample[
        (patents_sample['portal_index_id'].astype(str).str.contains(str(id)))]
    if filtered_data['publication_year'].dropna().count() != 0:
        patcita = float(filtered_data['npl_cite_count'].sum()) / float( filtered_data['publication_year'].dropna().count() )
    else:
        patcita = 0
    scores['patents citations'].loc[id] = patcita




sample_inventors_new = sample_inventors
sample_inventors_new.index = items
scores['Startup value (mm$)'] = sample_inventors_new['Startup value (mm$)']

scores['Total score'] = (scores.iloc[:,0:nn]).sum(axis=1)/nn #(scores.iloc[:,0]+scores.iloc[:,1]+scores.iloc[:,2]+scores.iloc[:,3]+scores.iloc[:,4]+5*scores.iloc[:,5])/nn #

for col in ['Total score','Startup value (mm$)']:
    min_val = scores[col].min()
    max_val = scores[col].max()
    scores[col] = (scores[col] - min_val) / (max_val - min_val)


scores = scores.round(3)


In [ ]:
scores

In [ ]:
scores_new = scores
scores_new.loc[scores_new['Total score'] < 0.15, 'Total score'] = 0

In [ ]:
fig, ax = plot.subplots(figsize=(6,4))

scores_new.plot(y='Total score', kind='bar', ax=ax, color='skyblue', position=0, width=0.8, label='Total Score',alpha=1)
scores_new.plot(y='Startup value (mm$)', kind='bar', ax=ax, color='salmon', position=0, width=0.6, label='Startup Value (mm$)',alpha=0.5)